# v0: jupy cannot work in pararell

In [1]:
import torch

tensor([0, 0, 0, 1, 1, 1, 1, 1, 1, 0])

In [35]:
def choice_mod(x,y,N):
    idx = torch.randint(0,x.size(0),(N,))
    return x[idx],y[idx]

In [36]:
x = torch.rand(100)
choice_mod(x,x,10,3)

(tensor([0.8364, 0.2831, 0.3012, 0.1584, 0.9977, 0.4090, 0.9608, 0.2144, 0.2144,
         0.0428]),
 tensor([0.8364, 0.2831, 0.3012, 0.1584, 0.9977, 0.4090, 0.9608, 0.2144, 0.2144,
         0.0428]))

In [4]:
perm = torch.randperm(x.size(0))

In [11]:
perm[:1000]

torch.Size([100])

In [2]:
import torch

In [48]:
torch.tensor([1,2,3,4,5],dtype=torch.double)

tensor([1., 2., 3., 4., 5.], dtype=torch.float64)

In [72]:
num_head = 5
center_id = int(num_head/2)
width = int((num_head-1)/2)
x = torch.zeros((num_head,num_head),dtype=torch.double)
x[:,center_id] = 1
for i in range(0,width):
    x[:i+1,i] = 1
    x[num_head-i-1:,num_head-i-1] = 1


tensor([[1., 1., 1., 0., 0.],
        [0., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 1., 1.]], dtype=torch.float64)

In [92]:
in_vec = torch.tensor([1,2,3,4,5],dtype=torch.double).view(5,1).expand(5,10)
in_vec.shape

torch.Size([10, 5])

In [89]:
in_vec.transpose(0,1).shape

torch.Size([10, 5])

In [91]:
(x@in_vec).shape

torch.Size([5, 10])

## quantile head

In [168]:
num_head = 5
center_id = 2

width = int((num_head-1)/2)
q_head = torch.zeros((num_head,num_head),dtype=torch.double)
q_head[center_id,:] = 1
for i in range(0,width):
    q_head[i,:i+1] = 1
    q_head[num_head-i-1,num_head-i-1:] = 1

model_out = torch.randn((10000,num_head),dtype=torch.double)
l = model_out[:,:center_id].clamp(max=1e-11)
m = model_out[:,center_id].unsqueeze(-1)
r = model_out[:,center_id+1:].clamp(max=1e-11)
(torch.cat((-1*l,m,r),dim=1)@q_head).shape

torch.Size([10000, 5])

# mean in model_out
- m not use unsqueeze

# to calc loss
- あとloss書いて組み込んだら終わり

In [188]:
quantiles = torch.tensor([i/(num_head+1) for i in range(1,num_head+1)],dtype=torch.double)

In [189]:
quantiles

tensor([0.1667, 0.3333, 0.5000, 0.6667, 0.8333], dtype=torch.float64)

tensor([-0.8333, -0.6667, -0.5000, -0.3333, -0.1667], dtype=torch.float64)

In [191]:
quan_out = (torch.cat((-1*l,m,r),dim=1)@q_head)

In [192]:
target_y = torch.randn((10000,1),dtype=torch.double)

In [193]:
error = (target_y.expand(target_y.shape[0],num_head) - quan_out)

In [207]:
loss = torch.max(error* quantiles ,error*(quantiles -1))

In [156]:
import numpy as np

In [162]:
np.log(3)

1.0986122886681098

In [148]:
q_head

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]], dtype=torch.float64)